<a href="https://colab.research.google.com/github/Nivratti/vedastr/blob/master/OCR_Media_Smart_vedastr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Media-Smart/vedastr

## Expectated Input:

1) Input image: Must be only single word image.

test image is located here:
https://drive.google.com/drive/folders/1Zr17Rcc7gTJ9SPV5phN6TwE4jr6shP2w



Input image will be resized to size 32, 100.

## Clone code

In [25]:
# !git clone https://github.com/Media-Smart/vedastr.git
!git clone https://github.com/Nivratti/vedastr.git

Cloning into 'vedastr'...
remote: Enumerating objects: 1573, done.
remote: Counting objects: 100% (174/174), done.
remote: Compressing objects: 100% (167/167), done.
remote: Total 1573 (delta 19), reused 96 (delta 5), pack-reused 1399
Receiving objects: 100% (1573/1573), 358.14 KiB | 11.94 MiB/s, done.
Resolving deltas: 100% (863/863), done.


## Move inside dir

In [ ]:
%cd "/content/vedastr"

/content/vedastr


## Download model

In [13]:
import gdown

# url = 'https://drive.google.com/uc?id=1YUOAU7xcrrsAtEqEGtI5ZD7eryP7Zr04'
# output = 'tps_resnet_bilstm_attn.pth'
# gdown.download(url, output, quiet=False)

## Case sensitive model
url = 'https://drive.google.com/uc?id=1bcKtEcYGIOehgPfGi_TqPkvrm6rjOUKR'
output = 'small_satrn.pth'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1bcKtEcYGIOehgPfGi_TqPkvrm6rjOUKR
To: /content/vedastr/small_satrn.pth
195MB [00:03, 60.7MB/s]


'small_satrn.pth'

## Install requirements

In [3]:
!pip install -r requirements.txt

     |████████████████████████████████| 2.1MB 13.2MB/s 
     |████████████████████████████████| 634kB 51.6MB/s 
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=bbd07b0b17dc8dc5bd90796a1bacb4c8537cde0f7a27fbe41e721fc0f8fe6a8b
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
  Created wheel for imgaug: filename=imgaug-0.2.6-cp37-none-any.whl size=654019 sha256=a7a60afde9d09ce33399322137af589fb6fe16d33821609b0f4ac1918d87e875
  Stored in directory: /root/.cache/pip/wheels/97/ec/48/0d25896c417b715af6236dbcef8f0bed136a1a5e52972fc6d0
Successfully built terminaltables imgaug
ERROR: bokeh 2.3.1 has requirement pillow>=7.1.0, but you'll have pillow 6.2.2 which is incompatible.
  Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully


## Modify config file -- Enable support for case sensitive letters

```python
# test_sensitive = False # default
test_sensitive = True
# test_character = '0123456789abcdefghijklmnopqrstuvwxyz' # default
test_character = '0123456789abcdefghijklmnopq' \
            'rstuvwxyzABCDEFGHIJKLMNOPQRS' \
            'TUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

```



In [21]:
#@title Update config file

%%writefile /content/vedastr/configs/small_satrn.py

###############################################################################
# 1. deploy

size = (32, 100)
mean, std = 0.5, 0.5

sensitive = True
character = '0123456789abcdefghijklmnopq' \
            'rstuvwxyzABCDEFGHIJKLMNOPQRS' \
            'TUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'  # need character

# test_sensitive = False # default
test_sensitive = True
# test_character = '0123456789abcdefghijklmnopqrstuvwxyz' # default
test_character = '0123456789abcdefghijklmnopq' \
            'rstuvwxyzABCDEFGHIJKLMNOPQRS' \
            'TUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

batch_max_length = 25

dropout = 0.1
n_e = 9
n_d = 3
hidden_dim = 256
n_head = 8
batch_norm = dict(type='BN')
layer_norm = dict(type='LayerNorm', normalized_shape=hidden_dim)
num_class = len(character) + 1
num_steps = batch_max_length + 1

deploy = dict(
    gpu_id='0,1,2,3',
    transform=[
        dict(type='Sensitive', sensitive=sensitive, need_character=character),
        dict(type='ToGray'),
        dict(type='Resize', size=size),
        dict(type='Normalize', mean=mean, std=std),
        dict(type='ToTensor'),
    ],
    converter=dict(
        type='AttnConverter',
        character=character,
        batch_max_length=batch_max_length,
        go_last=True,
    ),
    model=dict(
        type='GModel',
        need_text=True,
        body=dict(
            type='GBody',
            pipelines=[
                dict(
                    type='FeatureExtractorComponent',
                    from_layer='input',
                    to_layer='cnn_feat',
                    arch=dict(
                        encoder=dict(
                            backbone=dict(
                                type='GResNet',
                                layers=[
                                    ('conv',
                                     dict(type='ConvModule', in_channels=1, out_channels=int(hidden_dim / 2),
                                          kernel_size=3,
                                          stride=1, padding=1, norm_cfg=batch_norm)),
                                    ('pool', dict(type='MaxPool2d', kernel_size=2, stride=2, padding=0)),
                                    ('conv',
                                     dict(type='ConvModule', in_channels=int(hidden_dim / 2), out_channels=hidden_dim,
                                          kernel_size=3,
                                          stride=1, padding=1, norm_cfg=batch_norm)),
                                    ('pool', dict(type='MaxPool2d', kernel_size=2, stride=2, padding=0)),
                                ],
                            ),
                        ),
                        collect=dict(type='CollectBlock', from_layer='c2'),
                    ),
                ),
                dict(
                    type='SequenceEncoderComponent',
                    from_layer='cnn_feat',
                    to_layer='src',
                    arch=dict(
                        type='TransformerEncoder',
                        position_encoder=dict(
                            type='Adaptive2DPositionEncoder',
                            in_channels=hidden_dim,
                            max_h=100,
                            max_w=100,
                            dropout=dropout,
                        ),
                        encoder_layer=dict(
                            type='TransformerEncoderLayer2D',
                            attention=dict(
                                type='MultiHeadAttention',
                                in_channels=hidden_dim,
                                k_channels=hidden_dim // n_head,
                                v_channels=hidden_dim // n_head,
                                n_head=n_head,
                                dropout=dropout,
                            ),
                            attention_norm=layer_norm,
                            feedforward=dict(
                                type='Feedforward',
                                layers=[
                                    dict(type='ConvModule', in_channels=hidden_dim, out_channels=hidden_dim * 4,
                                         kernel_size=3, padding=1,
                                         bias=True, norm_cfg=None, activation='relu', dropout=dropout),
                                    dict(type='ConvModule', in_channels=hidden_dim * 4, out_channels=hidden_dim,
                                         kernel_size=3, padding=1,
                                         bias=True, norm_cfg=None, activation=None, dropout=dropout),
                                ],
                            ),
                            feedforward_norm=layer_norm,
                        ),
                        num_layers=n_e,
                    ),
                ),
            ],
        ),
        head=dict(
            type='TransformerHead',
            src_from='src',
            num_steps=num_steps,
            pad_id=num_class,
            decoder=dict(
                type='TransformerDecoder',
                position_encoder=dict(
                    type='PositionEncoder1D',
                    in_channels=hidden_dim,
                    max_len=100,
                    dropout=dropout,
                ),
                decoder_layer=dict(
                    type='TransformerDecoderLayer1D',
                    self_attention=dict(
                        type='MultiHeadAttention',
                        in_channels=hidden_dim,
                        k_channels=hidden_dim // n_head,
                        v_channels=hidden_dim // n_head,
                        n_head=n_head,
                        dropout=dropout,
                    ),
                    self_attention_norm=layer_norm,
                    attention=dict(
                        type='MultiHeadAttention',
                        in_channels=hidden_dim,
                        k_channels=hidden_dim // n_head,
                        v_channels=hidden_dim // n_head,
                        n_head=n_head,
                        dropout=dropout,
                    ),
                    attention_norm=layer_norm,
                    feedforward=dict(
                        type='Feedforward',
                        layers=[
                            dict(type='FCModule', in_channels=hidden_dim, out_channels=hidden_dim * 4, bias=True,
                                 activation='relu', dropout=dropout),
                            dict(type='FCModule', in_channels=hidden_dim * 4, out_channels=hidden_dim, bias=True,
                                 activation=None, dropout=dropout),
                        ],
                    ),
                    feedforward_norm=layer_norm,
                ),
                num_layers=n_d,
            ),
            generator=dict(
                type='Linear',
                in_features=hidden_dim,
                out_features=num_class,
            ),
            embedding=dict(
                type='Embedding',
                num_embeddings=num_class + 1,
                embedding_dim=hidden_dim,
                padding_idx=num_class,
            ),
        ),
    ),
    postprocess=dict(
        sensitive=test_sensitive,
        character=test_character,
    ),
)

###############################################################################
# 2.common

common = dict(
    seed=1111,
    logger=dict(
        handlers=(
            dict(type='StreamHandler', level='INFO'),
            dict(type='FileHandler', level='INFO'),
        ),
    ),
    cudnn_deterministic=True,
    cudnn_benchmark=True,
    metric=dict(type='Accuracy'),
)

###############################################################################
dataset_params = dict(
    batch_max_length=batch_max_length,
    data_filter=True,
    character=character,
)
test_dataset_params = dict(
    batch_max_length=batch_max_length,
    data_filter=False,
    character=test_character,
)

data_root = '../../../../dataset/str/data/data_lmdb_release/'

###############################################################################
# 3. test

batch_size = 256

# data
test_root = data_root + 'evaluation/'
test_folder_names = ['CUTE80', 'IC03_867', 'IC13_1015', 'IC15_2077',
                     'IIIT5k_3000', 'SVT', 'SVTP']
test_dataset = [dict(type='LmdbDataset', root=test_root + f_name,
                     **test_dataset_params) for f_name in test_folder_names]

test = dict(
    data=dict(
        dataloader=dict(
            type='DataLoader',
            batch_size=batch_size,
            num_workers=4,
            shuffle=False,
        ),
        dataset=test_dataset,
        transform=[
            dict(type='Sensitive', sensitive=test_sensitive, need_character=test_character),
            dict(type='ToGray'),
            dict(type='Resize', size=size),
            dict(type='Normalize', mean=mean, std=std),
            dict(type='ToTensor'),
        ],
    ),
    postprocess_cfg=dict(
        sensitive=test_sensitive,
        character=test_character,
    ),
)

###############################################################################
# 4. train

root_workdir = 'workdir'  # save directory

# data
train_root = data_root + 'training/'
# MJ dataset
train_root_mj = train_root + 'MJ/'
mj_folder_names = ['/MJ_test', 'MJ_valid', 'MJ_train']
# ST dataset
train_root_st = train_root + 'ST/'

train_dataset_mj = [dict(type='LmdbDataset', root=train_root_mj + folder_name)
                    for folder_name in mj_folder_names]
train_dataset_st = [dict(type='LmdbDataset', root=train_root_st)]

# valid
valid_root = data_root + 'validation/'
valid_dataset = dict(type='LmdbDataset', root=valid_root, **test_dataset_params)

train_transforms = [
    dict(type='Sensitive', sensitive=sensitive, need_character=character),
    dict(type='ToGray'),
    dict(type='ExpandRotate', limit=34, p=0.5),
    dict(type='Resize', size=size),
    dict(type='Normalize', mean=mean, std=std),
    dict(type='ToTensor'),
]

max_epochs = 6
milestones = [2, 4]  # epoch start from 0, so 2 means lr decay at 3 epoch, 4 means lr decay at the end of

train = dict(
    data=dict(
        train=dict(
            dataloader=dict(
                type='DataLoader',
                batch_size=batch_size,
                num_workers=4,
            ),
            sampler=dict(
                type='BalanceSampler',
                batch_size=batch_size,
                shuffle=True,
                oversample=True,
            ),
            dataset=dict(
                type='ConcatDatasets',
                datasets=[
                    dict(
                        type='ConcatDatasets',
                        datasets=train_dataset_mj,
                    ),
                    dict(
                        type='ConcatDatasets',
                        datasets=train_dataset_st,
                    )
                ],
                batch_ratio=[0.5, 0.5],
                **dataset_params,
            ),
            transform=train_transforms,
        ),
        val=dict(
            dataloader=dict(
                type='DataLoader',
                batch_size=batch_size,
                num_workers=4,
                shuffle=False,
            ),
            dataset=valid_dataset,
            transform=test['data']['transform'],
        ),
    ),
    optimizer=dict(type='Adam', lr=3e-4),
    criterion=dict(type='CrossEntropyLoss', ignore_index=num_class),
    lr_scheduler=dict(type='CosineLR',
                      iter_based=True,
                      warmup_epochs=0.1,
                      ),
    max_epochs=max_epochs,
    log_interval=10,
    trainval_ratio=2000,
    snapshot_interval=20000,
    save_best=True,
    resume=None,
)


Overwriting /content/vedastr/configs/small_satrn.py


## Inference on single word image

In [22]:
# download test image
url = 'https://drive.google.com/uc?id=1TdFP9xNVofQxYuA_RK0FUiukHS0R2tZI'
output = '/content/test.jpg'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1TdFP9xNVofQxYuA_RK0FUiukHS0R2tZI
To: /content/test.jpg
100%|██████████| 14.3k/14.3k [00:00<00:00, 8.23MB/s]


'/content/test.jpg'

In [23]:
# # !python tools/inference.py configs/tps_resnet_bilstm_attn.py checkpoint_path img_path
# !python tools/inference.py configs/tps_resnet_bilstm_attn.py "/content/tps_resnet_bilstm_attn.pth" "/content/t.png"

!python tools/inference.py configs/small_satrn.py "small_satrn.pth" "/content/test.jpg"

2021-04-29 05:43:39,086 - INFO - Use CPU
2021-04-29 05:43:39,086 - INFO - Set cudnn deterministic True
2021-04-29 05:43:39,086 - INFO - Set cudnn benchmark True
2021-04-29 05:43:39,086 - INFO - Set seed 1111
2021-04-29 05:43:39,087 - INFO - Build model
2021-04-29 05:43:39,096 - INFO - GResNet init weights
2021-04-29 05:43:39,505 - INFO - TransformerEncoder init weights
2021-04-29 05:43:39,973 - INFO - TransformerDecoder init weights
2021-04-29 05:43:40,007 - INFO - TransformerHead init weights
2021-04-29 05:43:40,043 - INFO - Load checkpoint from small_satrn.pth
2021-04-29 05:43:41,180 - INFO - predict string: ['RONALDO'] 	 of /content/test.jpg
